In [1]:
import os
from pathlib import Path
import pickle
import sys
sys.path.append('..')
import pandas as pd
import simplejson as json
from citylearn import CityLearn
from energy_models import Building

In [2]:
source_directory = Path('../data/Climate_Zone_5/')
target_directory = Path('dataset')
os.makedirs(target_directory, exist_ok=True)
filenames = ['rbc.pkl', 'marlisa.pkl']
space_use = {
    1: 'medium office',
    2: 'fast-food restaurant',
    3: 'standalone retail',
    4: 'strip mall retail',
    5: 'multifamily'
}
building_metadata = {}
# building_data_list = []
date_range = pd.date_range('2014-01-01 00:00:00', '2016-02-28 23:00:00', freq='H').tolist()\
    + pd.date_range('2016-03-01 00:00:00', '2017-12-31 23:00:00', freq='H').tolist()

In [3]:
# read env
for f in filenames:
    with (open(f, 'rb')) as openfile:
        simulation = pickle.load(openfile)
    
    env: CityLearn = simulation['env']
    agents = simulation['agents']
    actions = simulation['actions']
    rewards = simulation.get('rewards', None)

    for i, (b_id, b) in enumerate(env.buildings.items()):
        b: Building
        building_actions = [a[i] for a in actions]
        building_actions = pd.DataFrame(building_actions)
        action_index = 0

        # metadata
        building_metadata[b_id] = {
            'space_use':space_use[b.building_type],
            'systems': [{
                'id': f'cooling_device',
                'type': 'heat pump',
                'use': 'to meet cooling load and charge cooling storage.',
                'technical_efficiency': b.cooling_device.eta_tech,
                'nominal_power_kw': b.cooling_device.nominal_power,
                'target_cooling_temperature_c': b.cooling_device.t_target_cooling,
                'target_heating_temperature_c': b.cooling_device.t_target_cooling,
            }, {
                'id': f'dhw_device',
                'type': 'electric heater',
                'use': 'to meet domestic hot water heating load and charge domestic hot water storage.',
                'technical_efficiency': b.dhw_heating_device.efficiency,
                'nominal_power_kw': b.dhw_heating_device.nominal_power,
            }, {
                'id': f'cooling_storage',
                'type': 'thermal storage',
                'use': 'store cooling energy to offset building cooling load.',
                'capacity_kwh': b.cooling_storage.capacity,
                'technical_efficiency': b.cooling_storage.efficiency,
                'loss_coefficient': b.cooling_storage.loss_coef,
            }, {
                'id': f'dhw_storage',
                'type': 'thermal storage',
                'use': 'store heating energy to offset building domestic hot water heating load.',
                'capacity_kw': b.cooling_storage.capacity,
                'technical_efficiency': b.cooling_storage.efficiency,
                'loss_coefficient': b.cooling_storage.loss_coef,
            }, {
                'id': f'electrical_storage',
                'type': 'battery',
                'use': 'store electrical energy to offset building loads.',
                'capacity_kw': b.electrical_storage.capacity,
                'capacity_loss_coefficent': b.electrical_storage.capacity_loss_coef,
                'power_efficiency_curve': b.electrical_storage.power_efficiency_curve.tolist(),
                'capacity_power_curve': b.electrical_storage.capacity_power_curve.tolist(),
                'technical_efficiency': b.electrical_storage.efficiency,
                'loss_coefficient': b.electrical_storage.loss_coef,
            }, {
                'id': f'pv',
                'type': 'photovoltaic system',
                'use': 'self-generation to offset building loads. Excess generation is lost.',
                'capacity_kw': b.solar_power_capacity,
            }]
        }

        # time series
        agent = f.split('.')[0]
        data = {
            'agent': agent,
            'building_id': b_id,
            'timestamp': date_range[1:],
            'indoor_dry_bulb_temperature': b.sim_results['t_in'][1:],
            'indoor_relative_humidity': b.sim_results['rh_in'][1:],
            'cooling_load': b.sim_results['cooling_demand'][1:],
            'dhw_load': b.sim_results['dhw_demand'][1:],
            'plug_load': b.sim_results['non_shiftable_load'][1:],
            'pv_generation': b.sim_results['solar_gen'][1:],
            'net_electricity_consumption': b.net_electric_consumption,
            'net_electricity_consumption_no_storage': b.net_electric_consumption_no_storage,
            'net_electricity_consumption_no_storage_no_pv': b.net_electric_consumption_no_pv_no_storage,
            'cooling_device_electricity_consumption': b.electric_consumption_cooling,
            'cooling_device_cop': b.cooling_device.cop_cooling,
        }

        if b.dhw_heating_device.nominal_power > 0.1:
            data = {
                **data,
                'dhw_device_electricity_consumption': b.electric_consumption_dhw,
                'dhw_device_to_building_electricity_consumption': b.dhw_heating_device_to_building,
            }

        if b.cooling_storage.capacity > 0.1:
            data = {
                **data,
                'cooling_storage_net_electricity_consumption': b.electric_consumption_cooling_storage,
                'cooling_device_to_building_electricity_consumption': b.cooling_device_to_building,
                'cooling_storage_to_building_electricity_consumption': b.cooling_storage_to_building,
                'cooling_device_to_cooling_storage_electricity_consumption': b.cooling_device_to_storage,
                'cooling_storage_soc': b.cooling_storage_soc/b.cooling_storage.capacity,
                'cooling_storage_action': building_actions[action_index][1:]
            }
            action_index += 1
        else:
            pass

        if b.dhw_storage.capacity > 0.1:
            data = {
                **data,
                'dhw_storage_net_electricity_consumption': b.electric_consumption_dhw_storage,
                'dhw_storage_to_building_electricity_consumption': b.dhw_storage_to_building,
                'dhw_device_to_dhw_storage_electricity_consumption': b.dhw_heating_device_to_storage,
                'dhw_storage_soc': b.dhw_storage_soc/b.dhw_storage.capacity,
                'heating_storage_action': building_actions[action_index][1:]
            }
            action_index += 1
        else:
            pass

        if b.electrical_storage.capacity > 0.1:
            data = {
                **data,
                'electrical_storage_net_electricity_consumption': b.electrical_storage_electric_consumption,
                'electrical_storage_soc': b.electrical_storage_soc/b.electrical_storage.c0,
                'electrical_storage_action': building_actions[action_index][1:]
            }
            action_index += 1
        else:
            pass
        
        pd.DataFrame(data).to_csv(os.path.join(target_directory, f'{b_id}_{agent}.csv'), index=False)

# building_data = pd.concat(building_data_list, ignore_index=True, sort=False)
# del building_data_list
# building_data.to_csv(os.path.join(target_directory, 'simulation_output.csv'), index=False)

kwargs = {'ignore_nan':True,'sort_keys':False,'default':str,'indent':2}
with open(os.path.join(target_directory, 'building_metadata.json'), 'w') as f:
    json.dump(building_metadata, f, **kwargs)

In [ ]:
# carbon intensity
carbon_intensity_data = pd.read_csv(os.path.join(source_directory, 'carbon_intensity.csv'), parse_dates=['Datetime'])
carbon_intensity_data = carbon_intensity_data.rename(columns={
    'Datetime': 'timestamp',
    'kg_CO2/kWh': 'carbon_intensity'
})
carbon_intensity_data = carbon_intensity_data.iloc[1:,:]
carbon_intensity_data.to_csv(os.path.join(target_directory, 'carbon_intensity.csv'), index=False)

In [ ]:
# weather
weather_data = pd.read_csv(os.path.join(source_directory, 'weather_data.csv'))
weather_data.columns = [c.split(' [')[0].lower().replace(' ', '_') for c in weather_data.columns]
weather_data['timestamp'] = date_range
weather_data = weather_data[['timestamp'] + weather_data.columns[0:4].tolist()]
weather_data = weather_data.iloc[1:,:]
weather_data.to_csv(os.path.join(target_directory, 'weather.csv'), index=False)

In [ ]:
variable_metadata = pd.DataFrame([{'name': 'dhw_device_to_dhw_storage_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from dhw_device to dhw_storage.'},
 {'name': 'cooling_device_to_cooling_storage_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from cooling_device to cooling_storage.'},
 {'name': 'cooling_device_cop', 
 'unit': None, 
 'description': 'Cooling device (heat pump) Carnot refrigeration cycle coefficient of performance multiplied by technical efficiency.'},
 {'name': 'dhw_storage_to_building_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from dhw_storage to building.'},
 {'name': 'indoor_drybulb_temperature', 'unit': 'C', 'description': 'Building dry-bulb temperature.'},
 {'name': 'cooling_storage_action', 'unit': 'kWh/kWh_capacity', 'description': 'Control agent\'s selected proportion of cooling_storage capacity to charge or discharge cooling_storage by.'},
 {'name': 'dhw_load', 'unit': 'kWh', 'description': 'Domestic hot water load to be met by dhw_device and/or dhw_storage.'},
 {'name': 'direct_solar_radiation', 'unit': 'W/m^2', 'description': 'Direct solar radiation per unit area.'},
 {'name': 'dhw_device_to_building_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from dhw_device to building.'},
 {'name': 'dhw_storage_soc', 'unit': 'kWh/kWh_capacity', 'description': 'Energy stored in dhw_storage divided by dhw_storage capacity.'},
 {'name': 'cooling_load', 'unit': 'kWh', 'description': 'Space cooling load to be met by cooling_device and/or cooling_storage.'},
 {'name': 'net_electricity_consumption_no_storage_no_pv',
  'unit': 'kWh',
  'description': 'Building net electricity consumption in the absence of flexibility provided by storage devices and pv. Can be used as baseline/reference.'},
 {'name': 'dhw_storage_net_electricity_consumption',
  'unit': 'kWh',
  'description': 'dhw_storage net electricity consumption.'},
 {'name': 'diffuse_solar_radiation', 'unit': 'W/m^2', 'description': 'Diffuse solar radiation per unit area.'},
 {'name': 'cooling_storage_soc', 'unit': 'kWh/kWh_capacity', 'description': 'Energy stored in cooling_storage divided by cooling_storage capacity.'},
 {'name': 'net_electricity_consumption',
  'unit': 'kWh',
  'description': 'Building net electricity consumption.'},
 {'name': 'cooling_device_to_building_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from cooling device to building.'},
 {'name': 'building_id', 'unit': None, 'description': 'Unique building ID.'},
 {'name': 'cooling_storage_to_building_electricity_consumption',
  'unit': 'kWh',
  'description': 'Energy supply from cooling storage to building.'},
 {'name': 'heating_storage_action', 'unit': 'kWh/kWh_capacity', 'description': 'Control agent\'s selected proportion of dhw_storage capacity to charge or discharge dhw_storage by.'},
 {'name': 'plug_load', 'unit': 'kWh', 'description': 'Non-shiftable electricity load that must be met by grid and/or pv.'},
 {'name': 'indoor_relative_humidity', 'unit': '%', 'description': 'Building relative humidity.'},
 {'name': 'outdoor_relative_humidity', 'unit': '%', 'description': 'Outdoor relative humidity.'},
 {'name': 'outdoor_drybulb_temperature', 'unit': 'C', 'description': 'Outdoor dry-bulb temperature.'},
 {'name': 'net_electricity_consumption_no_storage',
  'unit': 'kWh',
  'description': 'Building net electricity consumption in the absence of flexibility provided by storage devices. Can be used as baseline/reference.'},
 {'name': 'dhw_device_electricity_consumption',
  'unit': 'kWh',
  'description': 'dhw_device electricity consumption in meeting dhw_load and dhw_storage load.'},
 {'name': 'cooling_storage_net_electricity_consumption',
  'unit': 'kWh',
  'description': 'cooling_storage net electricity consumption.'},
 {'name': 'timestamp', 'unit': 'YYYY-mm-dd HH:MM:SS', 'description': 'Variables datetime.'},
 {'name': 'cooling_device_electricity_consumption',
  'unit': 'kWh',
  'description': 'cooling_device electricity consumption in meeting cooling_load and cooling_storage load.'},
 {'name': 'pv_generation', 'unit': 'kW', 'description': 'PV solar generation.'},
  {'name':'electrical_storage_net_electricity_consumption', 'unit': 'kWh', 'description': 'electrical_storage net electricity consumption.'},
  {'name':'electrical_storage_soc', 'unit': 'kWh', 'description': 'Energy stored in electrical_storage divided by electrical_storage capacity.'},
  {'name':'electrical_storage_action', 'unit': 'kWh', 'description': 'Control agent\'s selected proportion of electrical_storage capacity to charge or discharge electrical_storage by.'},
  {'name':'carbon_intensity', 'unit': 'kg_CO2/kWh', 'description': 'Grid carbon intensity.'},
 ]).to_csv(os.path.join(target_directory, 'variable_metadata.csv'), index=False)
 

In [ ]:
# {k: len(v) for k, v in data.items()}